In [7]:
import matplotlib
import matplotlib.pyplot as plt

import os
import random
import io
import imageio
import glob
import scipy.misc
import numpy as np
from numpy import asarray
import pandas as pd
from six import BytesIO
from PIL import Image, ImageDraw, ImageFont
from IPython.display import display, Javascript
from IPython.display import Image as IPyImage

import tensorflow as tf

from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.utils import config_util
from object_detection.builders import model_builder
%matplotlib inline 

In [8]:
# Bounding Box Visualization pre-requesite

DATA_FOLDER = '../data/'

num_classes = 3

cloud_class_id = 0
water_class_id = 1
ground_class_id = 2

category_index = {
  cloud_class_id: {'id': cloud_class_id, 'name': 'cloud'},
  water_class_id: {'id': water_class_id, 'name': 'water'},
  ground_class_id: {'id': ground_class_id, 'name': 'ground'},
}

label_2_category = {'cloud':0,'water':1,'ground':2}

width, heigth = (2592, 1944) # these dimension are from astro pi default image

# Parsing XML annotation

In [9]:
import xml.etree.ElementTree as ET

# Get Image Annotation from XML Pascal VOC file
def get_img_annotations(img_xml_annotations_path):
  img_tree = ET.parse(img_xml_annotations_path)
  img_root = img_tree.getroot()
  labels_list = []
  bbox_list = []
  for child in img_root:
    if child.tag == "object":
      for element in child:
        if element.tag == "name":
          labels_list.append(element.text)
        if element.tag == "bndbox":
          bbox = []
          for coordinates in element:
            if coordinates.tag == "xmin":
              bbox.append(coordinates.text)
            if coordinates.tag == "ymin":
              bbox.append(coordinates.text)
            if coordinates.tag == "xmax":
              bbox.append(coordinates.text)
            if coordinates.tag == "ymax":
              bbox.append(coordinates.text)
          bbox_list.append(bbox)
  return labels_list, bbox_list

In [10]:
# Prepare Bounding Box so that it is normalized & invariant from image scale dimension
def get_prepared_bbox(bbox, width, heigth):
  for coord in bbox:
    xmin_norm=int(bbox[0])/width
    ymin_norm=int(bbox[1])/heigth
    xmax_norm=int(bbox[2])/width
    ymax_norm=int(bbox[3])/heigth
  prepared_bbox = [ymin_norm,xmin_norm,ymax_norm,xmax_norm] # Object Detection Viz utils expected format
  return prepared_bbox

# Data Prepaparation for Inference

In [18]:
# Get name of astropi Jpeg images
def get_images_name(data_folder):
  data = os.listdir(data_folder)
  images_file_list = [file for file in data if os.path.splitext(file)[1] != '.xml'] # remove xml file
  images_file_list.sort()
  return images_file_list

# Get name of astropi XML annotations
def get_bboxes_file_name(data_folder):
  data = os.listdir(data_folder)
  bbox_file_list = [file for file in data if os.path.splitext(file)[1] != '.jpg']
  bbox_file_list.sort()
  return bbox_file_list

# Get a List of astropi numpy images
def get_numpy_images(images_file,data_folder):
  np_images_list = []
  for image_file in images_file:
    img_pil = Image.open(data_folder+image_file)
    img_np = asarray(img_pil)
    np_images_list.append(img_np)
  return np_images_list

# Get a List of bboxes & classes for images
def get_gt_box_class(bbox_file_list,data_folder):
  gt_boxes_list = []
  classes_list = []
  for bboxes in bbox_file_list: 
    classes,gt_boxes=get_img_annotations(data_folder+bboxes)
    gt_boxes_np = np.array(gt_boxes,dtype=np.float32)
    gt_boxes_list.append(gt_boxes_np)
    classes_list.append(classes)
  return gt_boxes_list, classes_list
  
# Get a List of class_id for images
def get_class_id(classes_list): 
  classes_id_list = []
  for classes in classes_list:
    id_list  = []
    for label in classes:
      id = label_2_category[label]
      id_list.append(id)
    classes_id_list.append(id_list)
  return classes_id_list

In [20]:
# Grab Data from raw JPEG & XML files to lists
images_list = get_images_name(DATA_FOLDER)
np_images_list = get_numpy_images(images_list,DATA_FOLDER)
bboxes_file_list = get_bboxes_file_name(DATA_FOLDER)
gt_boxes_list, classes_list = get_gt_box_class(bboxes_file_list,DATA_FOLDER)
classes_id_list = get_class_id(classes_list)

In [21]:
def get_resized_img(size_tuple,img_tensor):
    resized_img = tf.image.resize(
        img_tensor, size_tuple, preserve_aspect_ratio=False,
        antialias=False, name=None
    )
    resized_img = tf.cast(resized_img,dtype=tf.uint8)
    return resized_img

def get_pad_resized_img(size,img_tensor):
    resized_img = tf.image.resize_with_pad(
        img_tensor, size,size,
        antialias=False
    )
    resized_img = tf.cast(resized_img,dtype=tf.uint8)
    return resized_img

# Inference from TFlite

In [22]:
tflite_model_path  = '../tflite/model.tflite'

In [23]:
def get_tflite_prepared_image(image_name):
  image_file = image_name + '.jpg'
  img_pil = Image.open(DATA_FOLDER+image_file)
  img_np = asarray(img_pil)
  img_tensor = tf.expand_dims(tf.convert_to_tensor(
    img_np, dtype=tf.float32), axis=0)
  preprocessed_img =  get_pad_resized_img(320,img_tensor)
  return preprocessed_img

In [24]:
def get_tflite_predictions(image_name,tflite_interpreter):
  # Prepare image
  preprocessed_img = get_tflite_prepared_image(image_name)
  input_tensor = preprocessed_img
  input_tensor = tf.cast(input_tensor,dtype=tf.float32)
  image_tflite_np = input_tensor.numpy()


  input_details = tflite_interpreter.get_input_details()
  output_details = tflite_interpreter.get_output_details() 

  # Prepare data for TFLite inference
  input_shape = input_details[0]['shape'] # tflite model input shape
  input_data = image_tflite_np.reshape(input_shape) # tflite model input data as numpy

  # This load input_data into tlfite interpreter
  interpreter.set_tensor(input_details[0]['index'], input_data)
  interpreter.invoke()

  # All useful Data from tflite inference: boxes, classes, scores
  boxes = interpreter.get_tensor(output_details[1]['index'])
  classes = interpreter.get_tensor(output_details[3]['index'])
  scores = interpreter.get_tensor(output_details[0]['index'])

  return boxes, classes, scores

In [ ]:
# Test image
img = 'zz_astropi_1_photo_189'

# Load the TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path=tflite_model_path)
interpreter.allocate_tensors()

# Get Prediction
boxes, classes, scores =  get_tflite_predictions(img,interpreter)

In [32]:
def post_process_df_prediction(prediction):
  prediction_list = []
  columns = ['ymin','xmin','ymax','xmax']
  prediction_df = pd.DataFrame(prediction[0][0],columns=columns)
  prediction_df['class'] = prediction[1][0].astype(np.int8)
  prediction_df['score'] = prediction[2][0]
  for index, row in prediction_df.iterrows():
    prediction_list.append(list(row))
  return prediction_list

In [37]:
def post_process_prediction(prediction):
  prediction_dico_list = []
  i  = 0 
  for (bbox,classe,score) in zip(prediction[0][0],prediction[1][0],prediction[2][0]):
    prediction_dico = {'id':i,'ymin':bbox[0],'xmin':bbox[1],'ymax':bbox[2],'xmax':bbox[3],'class':classe,'score':score}
    prediction_dico_list.append(prediction_dico)
    i = i+1
  return prediction_dico_list

In [38]:
prediction = get_tflite_predictions(img,interpreter)
post_proc_prediction = post_process_prediction(prediction)

In [ ]:
for e in post_proc_prediction:
  print(e)

# Inference from TPU

In [ ]:
from pycoral.utils import edgetpu
from pycoral.adapters import common
from pycoral.adapters import detect
from pycoral.utils.dataset import read_label_file

In [42]:
MODEL_FILE = '../tflite/efficientdet_bluelens_edgetpu.tflite'

In [43]:
# Load  TFLite model and allocate tensors.
def load_tflite_model(model_path):
    """Helper function to load the tflite object detection model
    Args:
        model_path (str): path of tflite for tpu model
    Returns:
        interpreter: the actual tflite interpreter/model
    """
    tflite_model_path  = model_path
    interpreter = edgetpu.make_interpreter(tflite_model_path)
    interpreter.allocate_tensors()
    return interpreter

In [44]:
def prepare_image(img_path,interpreter):
    """Prepare an image so that it's ready for interpreter inference
    Args:
        img_path (str): the path of the image to open
        interpreter(object): the tflite interpreter
    Returns:
        img (PIL): the opened image
        scale (tuple): ratio applied to image
    """
    image = Image.open(img_path)
    img, scale = common.set_resized_input(
    interpreter, image.size, lambda size: image.resize(size, Image.ANTIALIAS))
    return img,scale 

In [45]:
def do_inference(img_path,interpreter):
    """Do inference on image using object detection interpreter/model
    Args:
        img_path (str): the path of the image to open
        interpreter(object): the tflite interpreter
    Returns:
        obj_detection (object): the object detections done by interpreter/model
    """
    img,scale = prepare_image(img_path,interpreter)
    interpreter.invoke()
    obj_detection = detect.get_objects(interpreter, score_threshold=0.31, image_scale=scale)
    return obj_detection

In [ ]:
# Loading TFLite model
interpreter = load_tflite_model(str(DIR_PATH)+'/'+MODEL_FILE)
# Doing Prediction
tpu_prediction = do_inference(picture_name+'.jpg',interpreter)